# HOMEWORK 3  - INTERACTIVE VIZ

** Build a Choropleth map which shows intuitively (i.e., use colors wisely) how much grant money goes to each Swiss canton. To do so, you will need to use the provided TopoJSON file, combined with the Choropleth map example you can find in the Folium README file.**

**BONUS: using the map you have just built, and the geographical information contained in it, could you give a rough estimate of the difference in research funding between the areas divided by the Röstigraben?**

## 1. Get the canton to which each grant was assigned

Usual imports first.
- A INSTALLER AVEC pip install geopy

In [1]:
import pandas as pd
import numpy as np
import json
import os
from geopy.geocoders import Nominatim
from geopy.geocoders import GeoNames
from urllib import *

We load the grants and select a few useful attributes :
- The university and the institution will try to help us locate the canton to which each grant was assigned.
- The approved amount to know how much each canton received.

In [2]:
df_grants = pd.read_csv('P3_GrantExport.csv',sep=';')
df_grants = df_grants[[6,7,11,12,13]]
df_grants.head()

,Institution,University,Start Date,End Date,Approved Amount
0,NaN,Nicht zuteilbar - NA,01.10.1975,30.09.1976,11619.00
1,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,01.10.1975,30.09.1976,41022.00
2,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.03.1976,28.02.1985,79732.00
3,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,01.10.1975,30.09.1976,52627.00
4,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.01.1976,30.04.1978,120042.00


### 1. Extracting the canton from the university field
There are some issues in the data, that need to be formatted. First of all, some data need to be replaced with NaN, as they do not convey any information regarding the location of the grant that was assigned. We considered the following :
- 'Nicht zuteilbar - NA' -> Unavailable data
- 'NPO (Biblioth., Museen, Verwalt.) - NPO' -> No informations
- 'Weitere Institute - FINST' -> Other institutes
- 'Firmen/Privatwirtschaft - FP' -> Private institutions

There is also a second part of data that simply have no university field. The location of where those grants were assigned then shall be determined in the second part of the extraction, where we turn our sights to the institution field of the DataFrame

In [3]:
df_grants = df_grants.replace('Nicht zuteilbar - NA',np.nan)
df_grants = df_grants.replace('NPO (Biblioth., Museen, Verwalt.) - NPO',np.nan)
df_grants = df_grants.replace('Weitere Institute - FINST',np.nan)
df_grants = df_grants.replace('Firmen/Privatwirtschaft - FP',np.nan)

data_uni = df_grants['University'].unique()[1:]

data_uni[1:10]

array(['Universität Basel - BS', 'Université de Fribourg - FR',
       'Universität Zürich - ZH', 'Université de Lausanne - LA',
       'Universität Bern - BE',
       'Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL',
       'Université de Neuchâtel - NE', 'ETH Zürich - ETHZ',
       'Inst. de Hautes Etudes Internat. et du Dév - IHEID'], dtype=object)

In order to do a few less queries on the web to determine the location of our universities, we define two dictionnaries that we will extensively use to help us locate a university. The first one, `CANTON_DICT` associates each canton to its tag, that we will use on the map later on, but we can use it in the following way : if the key of this dict is present in the string describing the university, then we match it to the canton. It is very practical, as many universities are located in the capital of the canton, which often has the same name as the canton itself.

The second dict, `CAPITAL_DICT` contains pairs associating the capital of a canton to a canton, for those who do not have exactly the same name. We will use it the same way as `CANTON_DICT` : if the name of the capital appears in the string of the university, we match it to the canton it belongs to.

In [4]:
CANTON_DICT = {'Zürich':'ZH','Bern':'BE','Luzern':'LU','Uri':'UR','Schwyz':'SZ','Obwalden':'OW','Nidwalden':'NW',
               'Glarus':'GL','Zug':'ZG','Fribourg':'FR','Solothurn':'SO','Basel-Stadt':'BS','Basel-Landschaft':'BL',
               'Schaffhausen':'SH','Appenzell Ausserrhoden':'AR','Appenzell Innerrhoden':'AI','Sankt Gallen':'SG',
               'Graubünden':'GR','Aargau':'AG','Thurgau':'TG','Ticino':'TI', 'Vaud':'VD','Valais':'VS','Neuchâtel':'NE',
               'Genève':'GE','Jura':'JU','OUT': 'OUT'}
CAPITAL_DICT = {'Basel':'Basel-Stadt' ,'Lausanne':'Vaud', 'Sion':'Valais','Altdorf':'Uri','Sarnen':'Obwalden','Stans':'Nidwalden',
                'Liestal':'Basel-Landschaft','Herisau':'Appenzell Ausserrhoden',
                'Chur':'Graubünden','Aarau':'Aargau','Frauenfeld':'Thurgau','Bellinzona':'Ticino','Delémont':'Jura'}
#'Appenzell':'Appenzell Innerrhoden' excluded because the name of the city is partially in the name of two cantons

Let us now start with the canton extraction from the uni field. We will use two methods for that. 
- `extract_canton` will also be reused later on, tries to find, given a data_string containing a university/institution, the canton if belongs to. It does that in three steps :
    1. Tries to see if a canton appears in the data_string
    2. Tries to see if a capital which does not have the same name as the canton appears in the data_string
    3. Queries the Nominatim web service to try to locate the string. Here, if the response if `None`, we return `None`, and if the answer is not in Switzerland (its last field should contain `Svizra` in this case), then we also return `None`.

In [5]:
def extract_canton(data_string):
    """
        Fetchs from the web the location associated to a string. We use Nominatim at the moment. 
        The format of the output string (if it is not None) has the canton in the 6th output before the end.
        The canton is written in all the languages spoken in it, and we hence split and keep the name that comes first.
        @param data_string : the data from which we want to determine the canton
        @return canton : the name of the canton associated to the input data_string
    """
    for canton in CANTON_DICT:
        if(canton in str(data_string)):
            return canton
    for city,canton in CAPITAL_DICT.items():
        if(city in str(data_string)):
            return canton
    geolocator = Nominatim()#,username='test_056')
    location = geolocator.geocode(data_string)
    split_loc = str(location).split(', ')
    if (split_loc is not None) and (split_loc[-1] =='Svizra'):
        canton = str(location).split(', ')
        canton = canton[len(canton)-6]
        return canton.split(' - ')[0]   
    elif (split_loc != ['None']):
        print(data_string)
        print(split_loc)
        return 'OUT'
    else:
        return np.nan

- The second method, `extract_canton_from_uni`, will simply iterate on all the different universites of our DataFrame and call extract_canton. To get a higher chance of succeeding, we split the string at the `-` that separates the name and the acronym, and if the research was not successful with the full name, we pass the acronym to extract_canton as well. The full processing is done only once, and we store the result as a json file to load it again later on.

In [6]:
def extract_canton_from_uni(data_uni):
    """
        Extracts to which canton belongs a university and stored it into a json file (folder data).
        @param data_uni :           an array of strings describing universities or other institutions.
        @return university_dict :   a dict which associate to each university a canton if it was found
                                    or a None if nothing was found.
    """
    file = 'data/uni_canton_dict4.json'
    if(os.path.isfile(file)):
        print('Loading the data from json file')
        with open(file) as f:
            university_dict = json.load(f)
    else:
        print('Fetching the locations from the web')
        university_dict = dict()
        for uni in data_uni:
            # Splits the university string at the '-', which usually corresponds to the separation between the full name
            # and the acronym of the institution.
            uni_name_split = uni.split(' - ')
            extract_value = extract_canton(uni_name_split[0])
            if (extract_value is 'OUT') and (len(uni_name_split) > 1):
                # If the canton was not found in the name, then we try to extract it from the acronym string if it exists
                extract_value = extract_canton(uni_name_split[1])
            university_dict[uni] = extract_value
        with open(file, 'w') as f:
            json.dump(university_dict, f)
    return university_dict
university_canton = extract_canton_from_uni(data_uni)
#university_canton

Loading the data from json file


In [7]:
university_canton

{'AO Research Institute - AORI': 'OUT',
 'Allergie- und Asthmaforschung - SIAF': nan,
 'Berner Fachhochschule - BFH': 'Bern',
 'Biotechnologie Institut Thurgau - BITG': 'Thurgau',
 "Centre de rech. sur l'environnement alpin - CREALP": nan,
 'EPF Lausanne - EPFL': 'Vaud',
 'ETH Zürich - ETHZ': 'Zürich',
 'Eidg. Anstalt für Wasserversorgung - EAWAG': nan,
 'Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL': nan,
 'Eidg. Hochschulinstitut für Berufsbildung - EHB': nan,
 'Eidg. Material und Prüfungsanstalt - EMPA': nan,
 'Ente Ospedaliero Cantonale - EOC': 'Ticino',
 'Fachhochschule Kalaidos - FHKD': nan,
 'Fachhochschule Nordwestschweiz (ohne PH) - FHNW': nan,
 'Fachhochschule Ostschweiz - FHO': nan,
 'Facoltà di Teologia di Lugano - FTL': nan,
 'Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS': nan,
 'Forschungsanstalten Agroscope - AGS': nan,
 'Forschungsinstitut für Opthalmologie - IRO': nan,
 'Forschungsinstitut für biologischen Landbau - FIBL': 'Aargau',
 'Forschungskommission S

Let us now see the number of universities that were located and those who weren't.

In [8]:
print(len([place for place, value in university_canton.items() if not pd.isnull(value)]))
print(len([place for place, value in university_canton.items() if pd.isnull(value)]))

35
38


Sadly, this way only half of the universities were matches. Note that we did several tests and found that GeoNames leads a way poorer result than Nominatim (52 vs 37 not found), and prefiltering (trying to find the uni in the `CANTON_DICT` and `CAPITAL_DICT` dict) yields 37 not found against 40, but avoid queries on the web.

Let us now add a column to our grant DataFrame to find how many of the projects have been identified from their university name and fill this column with the canton tags from each canton

In [9]:
df_grants['Canton'] = np.nan
match_canton_uni = lambda uni: CANTON_DICT[university_canton[uni]] if (not pd.isnull(university_canton[uni])) else university_canton[uni]
df_grants.Canton = df_grants.University.loc[[uni in university_canton for uni  in df_grants.University]].apply(match_canton_uni)

In [10]:
df_grants.head(100)

,Institution,University,Start Date,End Date,Approved Amount,Canton
0,NaN,NaN,01.10.1975,30.09.1976,11619.00,NaN
1,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,01.10.1975,30.09.1976,41022.00,GE
2,Kommission für das Corpus philosophorum medii ...,NaN,01.03.1976,28.02.1985,79732.00,NaN
3,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,01.10.1975,30.09.1976,52627.00,BS
4,Schweiz. Thesauruskommission,NaN,01.01.1976,30.04.1978,120042.00,NaN
5,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,01.01.1976,31.12.1978,53009.00,FR
6,Institut für ökumenische Studien Université de...,Université de Fribourg - FR,01.01.1976,31.12.1976,25403.00,FR
7,Ostasiatisches Seminar Universität Zürich,Universität Zürich - ZH,01.10.1975,31.03.1977,47100.00,ZH
8,NaN,Université de Lausanne - LA,01.10.1975,31.03.1977,25814.00,VD
9,Laboratoire de Didactique et Epistémologie des...,Université de Genève - GE,01.10.1975,30.09.1978,360000.00,GE


In [11]:
print(len(df_grants[df_grants.Canton.isnull()]) / len(df_grants.Canton))
print(len(df_grants[df_grants.University.isnull() & df_grants.Institution.isnull()]) / len(df_grants.Canton))
df_grants[df_grants.Canton.isnull()]

0.315480936078413
0.06257718582438368


,Institution,University,Start Date,End Date,Approved Amount,Canton
0,NaN,NaN,01.10.1975,30.09.1976,11619.00,NaN
2,Kommission für das Corpus philosophorum medii ...,NaN,01.03.1976,28.02.1985,79732.00,NaN
4,Schweiz. Thesauruskommission,NaN,01.01.1976,30.04.1978,120042.00,NaN
11,Schweizerische Rechtsquellen c/o Universität Z...,NaN,01.10.1975,30.09.1978,862200.00,NaN
18,NaN,NaN,01.01.1976,31.12.1976,38152.00,NaN
24,NaN,NaN,01.01.1976,31.12.1979,56000.00,NaN
25,NaN,NaN,01.10.1975,30.09.1977,152535.00,NaN
26,Schweizerische Gesellschaft für Volkskunde,NaN,01.10.1975,30.09.1976,225000.00,NaN
27,Kuratorium Carl J. Burckhardt,NaN,01.10.1975,30.04.1978,179124.00,NaN
29,"Eidg. Forschungsanstalt für Wald, Schnee und L...","Eidg. Forschungsanstalt für Wald,Schnee,Land -...",01.10.1975,30.09.1978,445198.00,NaN


In [12]:
sum(df_grants.Canton.isnull()/len(df_grants.Canton))

0.31548093607847966

We see that only 30% of the cantons were found.

### 2. Extracting the canton from the institution field
Let us now address the potential problems we will encounter by displaying all the grants that have no universities associated.

In [13]:
df_grants.Institution.loc[df_grants.University.isnull()].value_counts()

Department of Chemistry University of Cambridge                                                                  42
Department of Chemistry Stanford University                                                                      29
Muséum d'Histoire Naturelle                                                                                      29
Commission nationale pour la publication des DDS                                                                 24
UNI: University of California   Berkeley USA                                                                     23
Gesellschaft für Schweizerische Kunstgeschichte                                                                  23
Redaktion des Schweizerdeutschen Wörterbuchs                                                                     22
Max-Planck-Institut für ausländisches und internationales Privatrecht                                            22
Schweizerische Vogelwarte                                               

We see that those institutions are mostly foreign universities or reasearch centers, that do not provide useful informations for our exercise. However, there are some swiss universities in the middle of it, so we will need to iterate through it and sort the institutions that will be kept or not.

In [14]:
df_institution = df_grants['Institution'].loc[df_grants['Canton'].isnull()]
df_institution = df_institution.unique()#head()
print(df_institution)
print(df_institution.shape)


INSTITUTION_EXCLUDED = []#['USA','GB','AUS','CDN','Stanford University','University of Cambridge','Paris','Essex','Shangai','Berlin']

[nan 'Kommission für das Corpus philosophorum medii aevi der SGG'
 'Schweiz. Thesauruskommission' ...,
 'Zoologisches Institut Universität Basel'
 'Abteilung für Neuropsychologie Zentrum für Kognitionswissenschaften Universität Bremen'
 'Departement Design Zürcher Hochschule der Künste ZHdK']
(9540,)


In [17]:
def extract_canton_from_institution(data_institution, complete = True):
    """
        Extracts to which canton belongs a institution and stored it into a json file (folder data). 
        As this data is very large, the queries to the website will most likely time out. We dump the the dictionnary 
        to a json file at each iteration and can restart from it with the parameter complete
        @param data_institution :   an array of strings describing institutions.
        @param complete         :   tells us if the process has been completed (everything saved to json file)
        @return institution_dict :  a dict which associate to each university a canton if it was found
                                    or a None if nothing was found.
    """
    institution_dict = dict()
    if(os.path.isfile('data/instit_canton_dict.json')):
        print('Loading the data from json file')
        with open('data/instit_canton_dict.json') as f:
            institution_dict = json.load(f)
        #print(institution_dict)
    if not(os.path.isfile('data/instit_canton_dict.json') and complete):
        print('Fetching the locations from the web')
        for index,instit in enumerate(data_institution):
            try:
                value = extract_canton(instit)
                    #break
            except :
                value = np.nan
                institution_dict[instit] = np.nan
                print(index,'- NOT FOUND <===== ',instit,)
            institution_dict[instit] = value
            if value is not np.nan:
                print(index,'-',value,'<=====',instit )
        with open('data/instit_canton_dict.json', 'w') as f:
            json.dump(institution_dict, f)
            excluded_bool = False
    return institution_dict
institution_canton = extract_canton_from_institution(df_institution[2000:2500],complete=False)

Loading the data from json file
Fetching the locations from the web
0 - NOT FOUND <=====  UNI: Bibliotheca Hertziana Max-Planck-Instit ut  Rom I
2 - Neuchâtel <===== Bibliothèque Office fédéral de la statistique
27 - NOT FOUND <=====  UNI: Università di Torino Dipartimento di St oria  Turin I
29 - Fribourg <===== UNI: Université de Fribourg Institut de jour nalisme et des communi- cations sociales Fri bourg CH
75 - Bern <===== Office féderal de la justice
78 - Zug <===== Kantonsschule Zug
79 - Valais <===== HES-SO Valais/Wallis
80 - Genève <===== Département de Biologie Moléculaire Faculté des Sciences Université de Genève
Institut für Kommunikationsforschung
['Institut für Kommunikationsforschung und Phonetik', 'Poppelsdorfer Allee', 'Südstadt', 'Stadtbezirk Bonn', 'Bonn', 'Regierungsbezirk Köln', 'Nordrhein-Westfalen', '53115', 'Deutschland']
91 - OUT <===== Institut für Kommunikationsforschung
101 - Bern <===== Büro für arbeits- und sozialpolitische Studien (Büro BASS), Bern
Sozialf

Elements 1 - 2482 sampled with Nominatim :

In [ ]:
#Gets the lengths of the dictionnary stored so we can know where to sample from (first number)
df_instit = df_grants.copy()
df_instit['Canton'] = np.nan
match_canton_insti = lambda uni: CANTON_DICT[institution_canton[uni]] if (not pd.isnull(institution_canton[uni])) else institution_canton[uni]
df_instit.Canton = df_instit.Institution.loc[[uni in institution_canton for uni  in df_instit.Institution]].apply(match_canton_insti)


In [ ]:
df_grants.head(12)

In [ ]:
df_instit.head(12)

In [ ]:
for index in df_grants.index :
 
 if df_grants.Canton.loc[index] != df_grants.Canton.loc[index] :
    df_grants.Canton.loc[index] = df_instit.Canton.loc[index]

df_grants.head(12)


In [ ]:
sum(df_grants.Canton.isnull()/len(df_grants.Canton))

In [ ]:
#print (df_grants[[df_grants['Approved Amount']=='data not included in P3']])
new = df_grants[df_grants['Approved Amount']!='data not included in P3']


In [ ]:

final = pd.DataFrame(columns=['prix'])
final.indexes=['Canton']
final

In [ ]:

for canton in CANTON_DICT :
 x = new[df_grants['Canton']==CANTON_DICT[canton]]['Approved Amount'].values
 x = x.astype(np.float)
 final.loc[CANTON_DICT[canton]]=[np.sum(x)/1e6]

final.head()
final = final.reset_index()
final.head()
final.to_csv('grants.csv')

In [ ]:
import folium
import pandas as pd

state_geo = r'data/ch-cantons.topojson.json'
state_grants = r'data/grants.csv'

state_data = pd.read_csv(state_grants)

print(state_data)

#Let Folium determine the scale
#ice_map = folium.Map(location=[-59.1759, -11.6016],
#                   tiles='Mapbox Bright', zoom_start=2)
#ice_map.choropleth(geo_path=state_geo, topojson='objects.cantons')
color_map = folium.Map(location=[48, -102], zoom_start=3)
color_map.choropleth(geo_path=state_geo,topojson='objects.cantons', data=state_data,
             columns=['index', 'prix'],
             key_on='id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='canton price (MCHF)')
color_map